In [ ]:
# get list of directories containing data to form datadir objects

import os
from dir_ext_set_getter import dir_ext_set_dict_builder
import itertools


os.chdir("/Users/jonathan/002_0_jono_data/")

print(os.getcwd())

To distinguish between single run directories and sequence directories, easiest way is to identify whether they contain a `.S` file. They appear to be semi-encoded files containing the sequence information.

In [ ]:
# seperate single runs and sequences into seperate lists

root_dir = "/Users/jonathan/002_0_jono_data/"

# iterate through the list

# print("START DIRECTORY")

root_dir_list = os.listdir(root_dir)

print(len(root_dir_list))

ext_set_dict = dir_ext_set_dict_builder(root_dir, root_dir_list)

print(len(ext_set_dict.keys()))

To ID the sequence directories, I need to identify unique characteristics between them and the single run data directories. Easiest way to survey the mass of dirs at hand is to form a list of each unique extension in the folder, then compare .S, .D, and others to identify if there is any variation, and what can be the identifier.

[This blog post](https://www.digitalocean.com/community/tutorials/get-unique-values-from-a-list-in-python) suggests using `set()`

Sets can be compared using set rules such as union, interseciton etc. THe most interesting thus far, in this [Medium](https://betterprogramming.pub/a-visual-guide-to-set-comparisons-in-python-6ab7edb9ec41) article, is `.symmetric_difference()` which returns the unique items in each set.

In [ ]:
for item in ext_set_dict.items():
    print(item)

In [ ]:
combo = itertools.combinations(ext_set_dict.keys(), 2)

combo = list(map(tuple, combo))

Now we have a list of all combos without replacement. Now to iterate through the list, access each set and compare them.

In [ ]:
# each pair contains a combination of two dict keys, formed from a combination without
# replacement of all items in the data directory. Iterating through each pair in the combo list
# then accessing the values of the dict (sets) to find items that they differ by, THEN identifying
# which of the pair the differences belong to.

# going to make a list of items belong to sequences and items belonging to data directories,
# then compare those lists. Probably makes the prior combinations superfluous but I will use this
# flow structure atm regardless.

single_run_item_list = []

sequence_item_list = []

for pair in combo[0:5]:
    difference = ext_set_dict[pair[0]].symmetric_difference(ext_set_dict[pair[1]])

    print(pair, "\n")
    if not difference:
        print("no difference\n")
    else:
        print(difference, "\n")
        for item in difference:
            if item in ext_set_dict[pair[0]]:
                print(item, "belongs to", pair[0])
            if item in ext_set_dict[pair[1]]:
                print(item, "belongs to", pair[1])

    print("\n###########################################\n")

A cursory glance at the output above hints that none of the .D dirs differ from each other (expected). We can check this with a quick modification

In [ ]:
difference_list = []


def combo_diff_printer(combo):
    for idx, pair in enumerate(combo):
        if ".D" in pair[0] and ".D" in pair[1]:
            print("###########################################\n")

            difference = ext_set_dict[pair[0]].symmetric_difference(
                ext_set_dict[pair[1]]
            )

            print("#", idx, pair, "\n")

            if not difference:
                print("no differences in subdirectories\n")
                continue

            else:
                print(difference, "\n")

                difference_list.append("item {} had a difference".format(idx))

                for item in difference:
                    if item in ext_set_dict[pair[0]]:
                        print(item, "belongs to", pair[0])
                    if item in ext_set_dict[pair[1]]:
                        print(item, "belongs to", pair[1])

                    print("\n")


print(difference_list)
if not difference_list:
    "no differences detected"

In the above, certain .D directories had large differences between them. After investigating the log files, I discovered that if they are an aborted run, they will not contain the following file types: 'acaml', 'txt', 'M', 'macaml', for example. This is a good way of differentiating between them. Of course, the best way is to delete the data file immediately. Straight off the bat, we can remove any .D subdirectories which do not contain those files.

I can use the any operator to check a subdir list for any of the extensions listed above. For example

How many .D dirs in the root dir?

In [ ]:
num_of_data_dirs = sum(".D" in item for item in root_dir_list)

print(num_of_data_dirs)

How many files in total?

In [ ]:
print(len(root_dir_list))

So, 8 sequences, 25 data dirs, 33 objects in total.

In [ ]:
list_of_aborted_data_dirs = []


def check_file_ext_in_subdir(item):
    subdir_path = os.path.join(os.getcwd(), item)

    subdir_list = os.listdir(subdir_path)

    #   if 'acaml' and 'txt' and 'M' and 'macaml' not in any(os.listdir(item_path)):

    # This condition returns 1 if acaml is found. i.e. completed run data.

    if any("acaml" in file for file in subdir_list):
        #        print(item)

        return 1

    else:
        return 0


count = 0

list_without_acaml = []


# iterating through the root dir list, targeting .D directories, returns the result (0 or 1) and counts how
# often a match occurs, then removes the item from the list if the match occurs.

for item in root_dir_list:
    if ".D" in item:
        result = check_file_ext_in_subdir(item)

        count += result

        if result == 0:
            print(item)

            list_without_acaml.append(item)

print("after filtering for 'acaml', list length is", len(list_without_acaml))

for item in list_without_acaml:
    if ".D" in item:
        print(os.listdir(os.path.join(os.getcwd(), item)))

print(count)

This result would lead me to believe that there are 3 aborted single runs in the root dir.

Now that I have some classifications, I would like to assemble everything into a data structure. Say a dict to a dataframe.

In [ ]:
a_dict = {"a": [1, 2, 3], "b": [4, 5, 6]}

import pandas as pd

df = pd.DataFrame(a_dict)

df

Just remove the aborted runs from the root_dir_list.

In [ ]:
print(len(root_dir_list))

for items in list_without_acaml:
    root_dir_list.remove(items)

print(len(root_dir_list))

Now we can get back to the business at hand, investigating what can separate .D dirs from sequence dirs.

In [ ]:
dict_set_without_aborts = dir_ext_set_dict_builder(root_dir, root_dir_list)

print(len(dict_set_without_aborts))

In [ ]:
#             if ".S" in sub_obj:

#                 sub_dir_file_path = os.path.join(file_path, obj)

#                 sequences.append(sub_dir_file_path)

#             if "SINGLE.B" in sub_obj:

#                   y = 1

# #                 print(sub_dir_file_path)

# #                 sub_dir_file_path = os.path.join(file_path, obj)

# #                 single_runs.append(sub_dir_file_path)

Now start assembling a table of the runs and the sequences, seperately. To do this programmatically, the filenames need to be cleaned up and made consistant. First offender is the date formats.

In [ ]:
for item in single_runs:
    split_item = item.split("/")

    print(split_item)

In [ ]:
# for item in single_runs:

#     item_basename = os.path.basename(item)

#     split = item_basename.split('_')

#     if "-" not in split[0]:

#         no_dash_format = split[0]

#         year = no_dash_format[:4]

#         month = no_dash_format[4:6]

#         day = no_dash_format[6:8]

#         dash_format = year + "-" + month + "-" + day

#         dash_format_item_basename = "_".join(split)

#         parent_dir_path = item.split("/")

#         print(parent_dir_path)

# #       os.rename(item, dash_format_item_basename)


2023-02-10-14:02 Tools that I need to continue the project are:

A function to form a dict of directories and their children and display in a readable format.

A function to form a combination size 2 without replacement of the keys of a given dictionary.

A function to describe the symmetric difference between each member of each combination.

Define each function from scratch individually, plug them together.

for a given FILTERED directory list:

    1. form a dict of sub directory names and children, return the dict list.
    2. form a combination size 2 from the keys of the list of dicts, return combination.
    3. display the differences.